In [1]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
# from ta.trend import SMAIndicator
# import pandas_datareader as pd_web
import yfinance as yf
import pandas as pd
import mplfinance as mpf
import json
import numpy as np
import math
import copy

np.seterr('raise')
np.warnings.filterwarnings('error', category=np.VisibleDeprecationWarning)

In [2]:
def relu(x):
    return np.where(x>0,x,0)

def softmax(x):
    x = np.exp(x - np.max(x))
    x[x==0] = 1e-15
    return np.array(x / x.sum())

def sigmoid(x):
    return 1/(1+np.exp(-x))

def plot_many(df:pd.DataFrame,*param):
    temp_df = df.copy()
    fig = go.Figure()
    for item in param:
        fig.add_trace(go.Scatter(x=temp_df.index,y=temp_df[item],name=item))
    fig.show()

def meashures(df:pd.DataFrame):
    _meashures={}
    temp_df = df.copy()
    # numero_de_anios=df.shape[0]/252
    # _meashures['cagr']=df['ret_acum_est'][-1]**(1/numero_de_anios)-1 #### CAGR
    # _meashures['volatility']=df['estrategia'].std()*np.sqrt(252)  #### VOLATILITY
    # _meashures['sharpe']= (_meashures['cagr']-0.022)/_meashures['volatility']  #### SHARPE
    # negvol= df[df['estrategia']<0]['estrategia'].std()*np.sqrt(252)  #### SORTINO
    # _meashures['sortino']= (_meashures['cagr']-0.022)/negvol  #### SORTINO

    # temp_df['max_ret_acum']= temp_df['ret_acum_est'].cummax()
    # temp_df['drawdown']= temp_df['max_ret_acum']-temp_df['ret_acum_est']
    # temp_df['drawdown_pct']=temp_df['drawdown']/temp_df['max_ret_acum']
    # _meashures['mdd']= temp_df['drawdown_pct'].max()
    # _meashures['calmar']=_meashures['cagr']/_meashures['mdd']

    _meashures['porcentaje superado']=(df['ret_acum_est'][df.shape[0]-1]-df['ret_acum'][df.shape[0]-1])/abs(df['ret_acum'][df.shape[0]-1]) 
    #### BETTER PERFORMING PERCENTAGE
    return _meashures['porcentaje superado']

In [3]:
class NeuralNet():
    def __init__(self, n_units=None, copy_network=None, var=0.02, episodes=50, max_episode_length=200):
        print("-",end='')
        # Testing if we need to copy a network
        if copy_network is None:
            # Saving attributes
            self.n_units = n_units
            # Initializing empty lists to hold matrices
            cls_weights = []
            vol_weights = []
            uni_weights = []
            cls_biases = []
            vol_biases = []
            uni_biases = []
            # Populating the lists
            for i in range(len(n_units)-1):
                cls_weights.append(np.random.normal(loc=0,scale=1,size=(n_units[i],n_units[i+1])))
                vol_weights.append(np.random.normal(loc=0,scale=1,size=(n_units[i],n_units[i+1])))
                uni_weights.append(np.random.normal(loc=0,scale=1,size=(2*n_units[-1],1)))
                cls_biases.append(np.zeros(n_units[i+1]))
                vol_biases.append(np.zeros(n_units[i+1]))
                uni_biases.append(np.zeros(1))
            # Creating dictionary of parameters
            self.params = {
                'cls_weights':cls_weights,
                'vol_weights':vol_weights,
                'uni_weights':uni_weights,
                'cls_biases':cls_biases,
                'vol_biases':vol_biases,
                'uni_biases':uni_biases
                }
        else:
            # Copying over elements
            self.n_units = copy_network.n_units
            self.params = { key:copy.deepcopy(copy_network.params[key]) for key in copy_network.params.keys() }
            for key in self.params.keys():
                self.params[key] = [x+np.random.normal(loc=0,scale=var,size=x.shape) for x in copy_network.params[key]]
            
    def act(self, X_cls, X_vol):
        # Grabbing weights and biases
        cls_weights = self.params['cls_weights']
        vol_weights = self.params['vol_weights']
        uni_weights = self.params['uni_weights']
        cls_biases = self.params['cls_biases']
        vol_biases = self.params['vol_biases']
        uni_biases = self.params['uni_biases']
        # First propgating inputs
        cls_a = relu((X_cls@cls_weights[0])+cls_biases[0])
        vol_a = relu((X_vol@vol_weights[0])+vol_biases[0])
        # Now propogating through every other layer
        for i in range(1,len(cls_weights)):
            cls_a = relu((cls_a@cls_weights[i])+cls_biases[i])
            vol_a = relu((vol_a@vol_weights[i])+vol_biases[i])
        uni_vector = []
        uni_vector.extend(cls_a)
        uni_vector.extend(vol_a)
        a = relu((uni_vector@uni_weights[i])+uni_biases[i])
        # Getting probabilities by using the softmax function
        probs = softmax(a)
        return 1 if probs[0]>0.5 else -1

    def predict(self,df:pd.DataFrame,time_limit=90):
        temp_df = df.copy()
        df_predictions=[]
        _scale = int(time_limit/self.n_units[0])
        for row in range(df.shape[0]):
            if row > time_limit:
                _X_cls = [ temp_df['Close'][row] ]
                _X_vol = [ temp_df['Volume'][row] ]
                _X_cls.extend([ temp_df['Close'][row] -temp_df['Close'][row-(_scale*index+1)]  for index in range(self.n_units[0]-1) ])
                _X_vol.extend([ temp_df['Volume'][row]-temp_df['Volume'][row-(_scale*index+1)] for index in range(self.n_units[0]-1) ])
                _X_cls = np.array(_X_cls)
                _X_cls = _X_cls/np.linalg.norm(_X_cls)
                _X_vol = np.array(_X_vol)
                _X_vol = _X_vol/np.linalg.norm(_X_cls)
                df_predictions.append( self.act( _X_cls, _X_vol ) )
            else:
                df_predictions.append(0)
        return df_predictions
        
    # Defining the evaluation method
    def evaluate(self,*df_argv,time_limit=90):
        print(".",end='')
        predictions= []
        for df in df_argv:
            temp_df = df.copy()
            df_predictions=[]
            _scale = int(time_limit/self.n_units[0])
            for row in range(df.shape[0]):
                if row > time_limit:
                    _X_cls = [ temp_df['Close'][row] ]
                    _X_vol = [ temp_df['Volume'][row] ]
                    _X_cls.extend([ temp_df['Close'][row] -temp_df['Close'][row-(_scale*index+1)]  for index in range(self.n_units[0]-1) ])
                    _X_vol.extend([ temp_df['Volume'][row]-temp_df['Volume'][row-(_scale*index+1)] for index in range(self.n_units[0]-1) ])

                    _X_cls = np.array(_X_cls)
                    _X_cls = _X_cls/np.linalg.norm(_X_cls)
                    _X_vol = np.array(_X_vol)
                    _X_vol = _X_vol/np.linalg.norm(_X_cls)

                    df_predictions.append( self.act( _X_cls, _X_vol ) )
                else:
                    df_predictions.append(0)
            temp_df['posicion']=df_predictions
            temp_df['retornos_del_instrumento'] = temp_df['Close'].pct_change()
            temp_df['estrategia'] = temp_df['posicion'].shift(1)*temp_df['retornos_del_instrumento']
            temp_df['ret_acum'] = temp_df['retornos_del_instrumento'].cumsum().apply(np.exp)
            temp_df['ret_acum_est'] = temp_df['estrategia'].cumsum().apply(np.exp)
            predictions.append( meashures(temp_df) )
        return np.amax( np.array(predictions) )

        

# Defining our class that handles populations of networks
class GeneticNetworks():
    # Defining our initialization method
    def __init__(self,datas, architecture=(4,16,2),population_size=50, generations=500,render_env=True, record=False,
                 mutation_variance=0.02,verbose=False,print_every=1,episodes=10,max_episode_length=200):
        # Creating our list of networks
        self.networks = [NeuralNet(architecture) for _ in range(population_size)]
        self.population_size = population_size
        self.generations = generations
        self.mutation_variance = mutation_variance
        self.verbose = verbose
        self.print_every = print_every
        self.fitness = []
        self.episodes = episodes
        self.max_episode_length = max_episode_length
        self.render_env = render_env
        self.record = record
        self.datas= datas
        
    # Defining our fiting method
    def fit(self):
        # Iterating over all generations
        for i in range(self.generations):
            # Doing our evaluations
            print()
            rewards = np.array([x.evaluate(*self.datas) for x in self.networks])
            # print(rewards)
            # Tracking best score per generation
            self.fitness.append(np.max(rewards))
            # Selecting the best network
            best_network = np.argmax(rewards)
            # Creating our child networks
            print()
            new_networks = [NeuralNet(copy_network=self.networks[best_network], var=self.mutation_variance, max_episode_length=self.max_episode_length) for _ in range(self.population_size-1)]
            # Setting our new networks
            self.networks = [self.networks[best_network]]+new_networks
            # # Printing output if necessary
            if self.verbose is True and (i%self.print_every==0 or i==0):
                print('Generation:',i+1,'| Highest Reward:',rewards.max().round(1),'| Average Reward:',rewards.mean().round(1))
        
        # Returning the best network
        self.best_network = self.networks[best_network]

In [4]:
criptos= ["TSLA"]
datas = [ yf.download(cripto,interval='1d',start='2020-01-01',end='2021-05-20') for cripto in criptos ]
# print(datas[0])
for index in range(len(criptos)):    
    datas[index] = datas[index].fillna(method='bfill')
    datas[index] = datas[index].fillna(method='ffill')

[*********************100%***********************]  1 of 1 completed


In [5]:
from time import time
start_time = time()
genetic_pop = GeneticNetworks(datas=datas,
                                architecture=(15,67,23,5),
                                population_size=200, 
                                generations=3,
                                episodes=15, 
                                mutation_variance=0.1,
                                max_episode_length=10000,
                                render_env=False,
                                verbose=True)
genetic_pop.fit()
print('Finished in',round(time()-start_time,3),'seconds')

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
........................................................................................................................................................................................................
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------Generation: 1 | Highest Reward: -0.6 | Average Reward: -0.6

........................................................................................................................................................................................................
-----------------------------------------------------------------------------------------------------------------------------------------

In [6]:
def estrategia(df:pd.DataFrame,plot_graphs=True,no_meashure=False):
    temp_df = df.copy()
    #### strategy implementation
    temp_df['posicion']= genetic_pop.best_network.predict(temp_df)

    #### benefit estimation
    # if(plot_graphs):
    #     plot_many(temp_df,"Close")
    temp_df['retornos_del_instrumento'] = temp_df['Close'].pct_change()
    temp_df['estrategia'] = temp_df['posicion'].shift(1)*temp_df['retornos_del_instrumento']
    temp_df['ret_acum'] = temp_df['retornos_del_instrumento'].cumsum().apply(np.exp)
    temp_df['ret_acum_est'] = temp_df['estrategia'].cumsum().apply(np.exp)

    #### meashures
    if(not no_meashure):
        if(plot_graphs):
            plot_many(temp_df,"posicion","ret_acum","ret_acum_est")
        def meashures(df:pd.DataFrame,plot_graphs=plot_graphs):
            meashures={}
            temp_df = df.copy()
            numero_de_anios=df.shape[0]/252
            meashures['cagr']=df['ret_acum_est'][-1]**(1/numero_de_anios)-1 #### CAGR
            meashures['volatility']=df['estrategia'].std()*np.sqrt(252)  #### VOLATILITY
            meashures['sharpe']= (meashures['cagr']-0.022)/meashures['volatility']  #### SHARPE
            negvol= df[df['estrategia']<0]['estrategia'].std()*np.sqrt(252)  #### SORTINO
            meashures['sortino']= (meashures['cagr']-0.022)/negvol  #### SORTINO

            temp_df['max_ret_acum']= temp_df['ret_acum_est'].cummax()
            temp_df['drawdown']= temp_df['max_ret_acum']-temp_df['ret_acum_est']
            temp_df['drawdown_pct']=temp_df['drawdown']/temp_df['max_ret_acum']
            meashures['mdd']= temp_df['drawdown_pct'].max()
            meashures['calmar']=meashures['cagr']/meashures['mdd']

            meashures['porcentaje superado']=(df['ret_acum_est'][df.shape[0]-1]-df['ret_acum'][df.shape[0]-1])/abs(df['ret_acum'][df.shape[0]-1]) #### BETTER PERFORMING PERCENTAGE
            if(plot_graphs):
                print("----------- data strategy meashures -------------")
                print(f"BEST :-> %{meashures['porcentaje superado']*100}\t\tporcentaje de mejoramiento del retorno por estrategia")
                print(f"CAGR :-> %{meashures['cagr']*100}\t\trentabilidad anual")
                print(f"VOL  :-> %{meashures['volatility']*100}\t\tvolatilidad anual")
                print(f"SHARP:->  {meashures['sharpe']}\t\t(R>1?)")
                print(f"SORTI:->  {meashures['sortino']}\t\t(R>1?)")
                print(f"MDD  :-> %{meashures['mdd']*100}\t\tMáxima caida relativa")
                print(f"CALM :->  {meashures['calmar']}\t\trentabilidad/riezgo")
                print("----------- data strategy meashures -------------\n----\n--\n-")
            return meashures
        return meashures(temp_df)
    else:
        meashures={}
        numero_de_anios=df.shape[0]/252
        meashures['cagr']=temp_df['ret_acum_est'][-1]**(1/numero_de_anios)-1 #### CAGR
        return meashures

In [7]:
data_index = 0
print(f"######## {criptos[data_index]} ######")
estrategia(datas[data_index])

######## TSLA ######


----------- data strategy meashures -------------
BEST :-> %-56.42167969881474		porcentaje de mejoramiento del retorno por estrategia
CAGR :-> %204.6619638985383		rentabilidad anual
VOL  :-> %62.49503366819831		volatilidad anual
SHARP:->  3.239648849114304		(R>1?)
SORTI:->  4.653609714625276		(R>1?)
MDD  :-> %34.685397134756265		Máxima caida relativa
CALM :->  5.900522433213204		rentabilidad/riezgo
----------- data strategy meashures -------------
----
--
-


{'cagr': 2.046619638985383,
 'volatility': 0.6249503366819831,
 'sharpe': 3.239648849114304,
 'sortino': 4.653609714625276,
 'mdd': 0.34685397134756263,
 'calmar': 5.900522433213204,
 'porcentaje superado': -0.5642167969881474}